## Annotation conversion


I need to train hand and face detection. For that, I'm converting a hand keypoint training set (that also happens to include face bouncing boxes).


Original dataset: http://domedb.perception.cs.cmu.edu/handdb.html

Converting head and hand keypoint dataset to head&hands bounding boxes.

Quick and dirty, sorry for that.

Issues, todos:
 - not all image contains both right and left hand [SOLVED]
 - image might contain multiple people, annotation in separate files
 - number of keys: 9 the first labels, 7 for the latter. E.g. image file name is missing...

### Darknet label format


"Now we need to generate the label files that Darknet uses. Darknet wants a .txt file for each image with a line for each ground truth object in the image that looks like:

```<object-class> <x> <y> <width> <height>```

Where x, y, width, and height are relative to the image's width and height."  https://pjreddie.com/darknet/yolo/

In [ ]:
%matplotlib notebook
import os
from matplotlib import pyplot as plt
import json
import numpy as np

In [ ]:
inpath = "/home/ubuntu/data/hand_labels/manual_train/"
files = sorted([f for f in os.listdir(inpath) if f.endswith('.json')])
files_left = sorted([f for f in os.listdir(inpath) if f.endswith('l.json')])
files_right = sorted([f for f in os.listdir(inpath) if f.endswith('r.json')])

In [ ]:
# both hands are not available for all samples

i = 0
f = files[i];

dat_left, dat_right = None, None

if (os.path.exists(inpath+f[0:-6]+'l.json')):
    f_left = f[0:-6]+'l.json'
    with open(inpath+f_left, 'r') as fid:
        dat_left = json.load(fid)
    
if (os.path.exists(inpath+f[0:-6]+'r.json')):
    f_right = f[0:-6]+'r.json'
    with open(inpath+f_right, 'r') as fid:
        dat_right = json.load(fid)

In [ ]:
def get_boundingbox(points_dat):
    points = np.array(points_dat)
    x1 = points[:, 0].min()
    x2 = points[:, 0].max()
    y1 = points[:, 1].min()
    y2 = points[:, 1].max()
    return [x1, x2, x2, x1, x1], [y1, y1, y2, y2, y1]

def get_yolo_params(points_dat, width, height):
    points = np.array(points_dat)
    x1 = points[:, 0].min()
    x2 = points[:, 0].max()
    y1 = points[:, 1].min()
    y2 = points[:, 1].max()
    
    x = (x1 + x2) / 2 / width
    y = (y1 + y2) / 2 / height
    
    rel_width = (x2 - x1) / width
    rel_height = (y2 - y1) / height
    
    return x, y, rel_width, rel_height


In [ ]:
im = plt.imread(inpath+f_left[0:-5]+'.jpg')

fig = plt.figure(0)
ax = fig.subplots(1)
ax.imshow(im)
height, width = im.shape[0], im.shape[1]
head_drawn = False
if dat_left is not None:
    head_x, head_y = get_boundingbox(dat_left['head_box'])
    print('Head: ', get_yolo_params(dat_left['head_box'], width, height))
    hand_left_x, hand_left_y = get_boundingbox(dat_left['hand_pts'])
    print('Left hand: ', get_yolo_params(dat_left['hand_pts'], width, height))

    head_drawn = True
    ax.plot(head_x, head_y)
    ax.plot(hand_left_x, hand_left_y)

    
if dat_right is not None: 
    hand_right_x, hand_right_y = get_boundingbox(dat_right['hand_pts'])
    print('Right hand: ', get_yolo_params(dat_right['hand_pts'], height, width))

    ax.plot(hand_right_x, hand_right_y)
    if not head_drawn:
        head_x, head_y = get_boundingbox(dat_right['head_box'])
        ax.plot(head_x, head_y)
        






### For now, only create annotation for images with a single person

This is still 338 images for train.

In [ ]:
files_restricted = sorted([f for f in os.listdir(inpath) if f.endswith('_01_l.json')])
len(files_restricted)


data_dir = "/home/ubuntu/data/hand_labels/"

test_in_dir = "/home/ubuntu/data/hand_labels/manual_test/"

train_out_dir = '/home/ubuntu/data/hand_labels/yolo_train/'
test_out_dir = '/home/ubuntu/data/hand_labels/yolo_train/'

for split in ['train', 'test']:
    print('Converting {} ...'.format(split))
    
    in_dir = data_dir + 'manual_' + split + '/'
    out_dir = data_dir + 'yolo_' + split + '/'
    
    files_restricted = sorted([f for f in os.listdir(in_dir) if f.endswith('_01_l.json')])
    print('Converting ', len(files_restricted), ' samples.')
    
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    image_paths = []

    for idx, f in enumerate(files_restricted[:]):

        name = '{:05d}'.format(idx)
        print(name)

        dat_left, dat_right = None, None

        if (os.path.exists(in_dir+f[0:-6]+'l.json')):
            f_left = f[0:-6]+'l.json'
            with open(in_dir+f_left, 'r') as fid:
                dat_left = json.load(fid)

        if (os.path.exists(in_dir+f[0:-6]+'r.json')):
            f_right = f[0:-6]+'r.json'
            with open(in_dir+f_right, 'r') as fid:
                dat_right = json.load(fid)

        has_left = dat_left is not None

        im = plt.imread(in_dir+(f_left[0:-5] if has_left else f_right[0:-5])+'.jpg')
        plt.imsave(os.path.join(out_dir, name + '.jpg'), im)
        image_paths.append(os.path.join(out_dir, name + '.jpg'))

        height, width = im.shape[0], im.shape[1]

        with open(os.path.join(out_dir, name + '.txt'), 'w') as label_file:
            if has_left:
                hx, hy, hw, hh = get_yolo_params(dat_left['head_box'], width, height)
                lx, ly, lw, lh = get_yolo_params(dat_left['hand_pts'], width, height)
                print("1 {} {} {} {}".format(hx, hy, hw, hh), file=label_file)
                print("2 {} {} {} {}".format(lx, ly, lw, lh), file=label_file)


            if dat_right is not None:
                if not has_left:
                    hx, hy, hw, hh = get_yolo_params(dat_right['head_box'], width, height)
                    print("1 {} {} {} {}".format(hx, hy, hw, hh), file=label_file)
                rx, ry, rw, rh = get_yolo_params(dat_right['hand_pts'], width, height)
                print("2 {} {} {} {}".format(rx, ry, rw, rh), file=label_file)

    with open(os.path.join(out_dir, '..', 'yolo_{}.txt'.format(split)), 'w') as f:
        for line in image_paths:
            print(line, file=f)



In [ ]:
has_left = True
has left?

In [ ]:
os.listdir(out_dir)